## ALOS classification of wetlands on GEE 
#### Method overview:
1. Evaluate coverage of ALOS scenes; this step is currently coded in R.
    - Download metadata of Radio-metricly corrected (RTC) scenes (*DONE*)
    - Map number of overlapping scenes.
    - Map seasonal coverage of SWAMPS-GLWD inundation.
    
2. Prepare training data as points 
    - Filter NWI for the US, by image year with the metadata polygon. Copy the water regime codes to the polygon attribute.
    - Africover
    
3. Gather scene granule over the training sites.
    - Lump small overlap polygons to reduce number of polygons to process.
    
4. Ingest ALOS scenes (GeoTIFFs) in GEE; with a batch script of Command Line tools.
    - First, in Google Cloud Storage with the command line. Ensure the bucket is public, so that GEE can access it.
    - Then, import the image to GEE.
    - The images transferred are:  sigma-naught, incidence angle.

5. Classify stack of ALOS images as wetland/non-wetland.
    - Fit harmonic model on full time series
    - Train classifier on: average EVI, parameters of SAR harmonic, topography
    - Test prepare in JS, then convert to Python.

6. Classify individual scenes of ALOS based on threshold.
    - Stack these classified scenes, to get average inundation period.

7. Save the classified image(s)
    - To asset, then save to drive, then download.



In [2]:
# set directory
import os
os.chdir("C:/Users/efluet/Dropbox/alos/data/asf_scenes")

In [7]:

# Prep training data
# Loop through NWI metadata grids.
execfile("./training_data/nwi_training_sites.py")

# Upload the shapefile to GEE
execfile("./gcs/fcs_transfer_from_local_to_gee.py")


DistributionNotFound: The 'google-cloud-core<2.0dev,>=1.0.0' distribution was not found and is required by google-cloud-storage

In [ ]:
# Download the images; passing the ROI poly coordinates.
# Select ALOS images to upload
# Get the granule list of images intersecting the zone.
execfile("./dl_alos_image/dl_alos_images.py")



In [3]:
# Unzip the HH image from each scene/Granule.
execfile("../../scripts/python/unzip_alos_hh.py")  # , globals())

# Upload the _HH image to GEE


NameError: name 'execfile' is not defined

In [ ]:
# Run classification on GEE
# Use JS for development/visualization.
execfile("./js/classify_image_wlandsat_date_filter.js")
# Then convert to a Python API script that exports the classified output.


In [1]:
# Interactive visualization in the notebook.
#from __future__ import print_function
from ipyleaflet import * # Map, basemaps, basemap_to_tiles

m = Map(center=(52, 10), zoom=8, basemap=basemaps.Hydda.Full)
m

C:\Users\efluet\Anaconda3\lib\site-packages\dask\config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}
C:\Users\efluet\Anaconda3\lib\site-packages\distributed\config.py:20: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  defaults = yaml.load(f)


Map(basemap={'url': 'http://{s}.tile.openstreetmap.se/hydda/full/{z}/{x}/{y}.png', 'max_zoom': 18, 'attributio…

In [12]:
import os
print(os.environ[r"$env:GOOGLE_APPLICATION_CREDENTIALS='C:\Users\efluet\Dropbox\alos\scripts\key\My First Project-e3eea9f7abba.json'"])
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r'C:\Users\efluet\Dropbox\alos\scripts\key\My First Project-e3eea9f7abba.json'


KeyError: "$env:GOOGLE_APPLICATION_CREDENTIALS='C:\\Users\\efluet\\Dropbox\\alos\\scripts\\key\\My First Project-e3eea9f7abba.json'"

In [22]:
# Imports the Google Cloud client library
from google.cloud import storage

# Instantiates a client
storage_client = storage.Client()

# The name for the new bucket
bucket_name = 'alos-scenes-bucket'

# Creates the new bucket
bucket = storage_client.create_bucket(bucket_name)
print('Bucket {} created.'.format(bucket.name))



DistributionNotFound: The 'google-cloud-core<2.0dev,>=1.0.0' distribution was not found and is required by google-cloud-storage

In [18]:
import environ
env = environ.Env.read_env() # reading .env file
SECRET_KEY = env('SECRET_KEY')
# earthengine upload image --asset_id=users/username/asset_id gs://bucket/image.tif


ModuleNotFoundError: No module named 'environ'

In [8]:
# /------------------------------------------------------------------------------------#
#/ Import ALOS GeoTIFFs
import os
import ee
ee.Initialize()

# Use the Command Line Tool: https://developers.google.com/earth-engine/command_line#upload
# earthengine upload image --asset_id=users/username/asset_id gs://bucket/image.tif

# Print list of assets
ls = os.system('earthengine ls users/etiennefluet')
print(ls)
import commands
print(commands.getoutput('earthengine ls users/etiennefluet'))


i = ee.Image("users/etiennefluet/AP_09556_FBS_F0590_RT2_HH")#.clip(area_of_interest)
print(i)

0


ModuleNotFoundError: No module named 'commands'

In [ ]:
# Import vector point data
# Export to KML Export.table.toDrive())
# Then upload to Fusion Table or export to Earth Engine asset
# Export.table.toAsset()
# Load training points. The numeric property 'class' stores known labels.
var points = ee.FeatureCollection('ft:10X7SUjDTiFJDyIA58zLcptK8pwBwjj1BV12SQOgJ')
    .remap([1, 2], [0, 1], 'class');

# Overlay the points on the imagery to get training.
var training = image.sampleRegions({
  collection: points,
  properties: ['class'],
  scale: 30
});

# Train a CART classifier with default parameters.
var trained = ee.Classifier.cart().train(training, 'class', bands);

# Classify the image with the same bands used for training.
var classified = image.select(bands).classify(trained);

# Display the inputs and the results.
Map.centerObject(image, 10);
Map.addLayer(image, {bands: ['B4', 'B3', 'B2'], max: 0.4}, 'image');
Map.addLayer(classified, {min: 0, max: 1, palette: ['00FF00', 'FF0000']},
  'classification');





To Do list:
__Tutorials available here:__
- https://geoscripting-wur.github.io/Earth_Engine/#WUR_Geoscripting_
- https://github.com/google/earthengine-api/blob/master/python/examples/ipynb/TF_demo1_keras.ipynb
1. Make map with ipyleaflet
2. upload Biomass Data

